# 01 — Data Validation & Merge Pipeline
## HumanForYou — Employee Attrition Prediction

---

### Objective

Establish a **reproducible and traceable workflow** to:
1. Validate the integrity of all 4 source datasets (general_data, employee_survey, manager_survey, badge data)
2. Check schema consistency, missing values, and data quality
3. Merge into a single analysis-ready DataFrame
4. Export the clean dataset for downstream notebooks

> **Adapted from** the pipeline architecture of a previous YOLO detection project — same rigor, different domain.

## Section 1: Configuration and Environment Setup

In [1]:
# ==============================================================================
# CONFIGURATION — Centralized paths and parameters
# ==============================================================================

import os
import sys
import warnings
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

# Only suppress expected warnings, not real errors
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

# --- Path Configuration ---
# Detect project root reliably (works regardless of CWD)
_cwd = Path.cwd()
if (_cwd / "data" / "raw").exists():
    PROJECT_ROOT = _cwd                          # CWD is project root
elif (_cwd.parent / "data" / "raw").exists():
    PROJECT_ROOT = _cwd.parent                   # CWD is notebooks/
else:
    raise FileNotFoundError(
        "Cannot find project root: 'data/raw/' not found in CWD or parent. "
        "Run this notebook from the project root or notebooks/ directory."
    )

DATA_DIR   = str(PROJECT_ROOT / "data" / "raw")
OUTPUT_DIR = str(PROJECT_ROOT / "outputs")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Source files declaration
FILES = {
    "general":         os.path.join(DATA_DIR, "general_data.csv"),
    "employee_survey": os.path.join(DATA_DIR, "employee_survey_data.csv"),
    "manager_survey":  os.path.join(DATA_DIR, "manager_survey_data.csv"),
    "in_time":         os.path.join(DATA_DIR, "in_time.csv"),
    "out_time":        os.path.join(DATA_DIR, "out_time.csv"),
}

EMPLOYEE_ID_COL = "EmployeeID"

print(f"Configuration loaded — {datetime.now():%Y-%m-%d %H:%M}")
print(f"Project root   : {PROJECT_ROOT}")
print(f"Data directory  : {DATA_DIR}")
print(f"Output directory: {OUTPUT_DIR}")

Configuration loaded — 2026-02-20 12:16
Project root   : C:\Users\yanis\Documents\CESI\A5\AI Project\HumanForYou
Data directory  : C:\Users\yanis\Documents\CESI\A5\AI Project\HumanForYou\data\raw
Output directory: C:\Users\yanis\Documents\CESI\A5\AI Project\HumanForYou\outputs


## Section 2: File Existence & Schema Validation

**Purpose**: Verify every source file exists, is non-empty, and has the expected columns.

In [2]:
# ==============================================================================
# FILE EXISTENCE CHECK
# ==============================================================================

def validate_file(name, path):
    """Check file exists and is non-empty. Return row/col counts."""
    if not os.path.isfile(path):
        print(f"  [FAIL] {name}: file not found at {path}")
        return None
    size_kb = os.path.getsize(path) / 1024
    df = pd.read_csv(path)
    print(f"  [OK]   {name:20s} — {df.shape[0]:>5} rows × {df.shape[1]:>3} cols  ({size_kb:,.0f} KB)")
    return df

print("=" * 65)
print("FILE VALIDATION")
print("=" * 65)

raw = {}
for key, path in FILES.items():
    result = validate_file(key, path)
    if result is not None:
        raw[key] = result

print(f"\nLoaded {len(raw)}/{len(FILES)} files successfully.")

FILE VALIDATION
  [OK]   general              —  4410 rows ×  24 cols  (537 KB)
  [OK]   employee_survey      —  4410 rows ×   4 cols  (51 KB)
  [OK]   manager_survey       —  4410 rows ×   3 cols  (42 KB)
  [OK]   in_time              —  4410 rows × 262 cols  (22,741 KB)
  [OK]   out_time             —  4410 rows × 262 cols  (22,741 KB)

Loaded 5/5 files successfully.


In [3]:
# ==============================================================================
# SCHEMA VALIDATION — Expected columns
# ==============================================================================

EXPECTED_SCHEMAS = {
    "general": [
        "Age", "Attrition", "BusinessTravel", "Department", "DistanceFromHome",
        "Education", "EducationField", "EmployeeCount", "EmployeeID", "Gender",
        "JobLevel", "JobRole", "MaritalStatus", "MonthlyIncome",
        "NumCompaniesWorked", "Over18", "PercentSalaryHike", "StandardHours",
        "StockOptionLevel", "TotalWorkingYears", "TrainingTimesLastYear",
        "YearsAtCompany", "YearsSinceLastPromotion", "YearsWithCurrManager"
    ],
    "employee_survey": ["EmployeeID", "EnvironmentSatisfaction", "JobSatisfaction", "WorkLifeBalance"],
    "manager_survey":  ["EmployeeID", "JobInvolvement", "PerformanceRating"],
}

print("SCHEMA VALIDATION")
print("=" * 65)
for key, expected_cols in EXPECTED_SCHEMAS.items():
    actual = list(raw[key].columns)
    missing = set(expected_cols) - set(actual)
    extra   = set(actual) - set(expected_cols)
    status = "OK" if not missing else "FAIL"
    print(f"  [{status}] {key}")
    if missing:
        print(f"         Missing columns: {missing}")
    if extra:
        print(f"         Extra columns  : {extra}")

SCHEMA VALIDATION
  [OK] general
  [OK] employee_survey
  [OK] manager_survey


## Section 3: Data Quality Audit

**Purpose**: For each dataset, check missing values, duplicates, constant columns, and basic statistics.

In [4]:
# ==============================================================================
# MISSING VALUES REPORT
# ==============================================================================

def missing_report(df, name):
    """Print missing value summary for a DataFrame."""
    total = df.isnull().sum()
    pct   = (total / len(df) * 100).round(2)
    report = pd.DataFrame({"missing": total, "pct": pct})
    report = report[report["missing"] > 0].sort_values("pct", ascending=False)
    if report.empty:
        print(f"  {name}: No missing values ✓")
    else:
        print(f"  {name}: {len(report)} column(s) with missing values")
        for col, row in report.iterrows():
            print(f"    → {col}: {int(row['missing'])} ({row['pct']:.1f}%)")
    return report

print("MISSING VALUES AUDIT")
print("=" * 65)
missing_reports = {}
for key, df in raw.items():
    if key not in ("in_time", "out_time"):  # badge data handled separately
        missing_reports[key] = missing_report(df, key)

MISSING VALUES AUDIT
  general: 2 column(s) with missing values
    → NumCompaniesWorked: 19 (0.4%)
    → TotalWorkingYears: 9 (0.2%)
  employee_survey: 3 column(s) with missing values
    → WorkLifeBalance: 38 (0.9%)
    → EnvironmentSatisfaction: 25 (0.6%)
    → JobSatisfaction: 20 (0.5%)
  manager_survey: No missing values ✓


In [5]:
# ==============================================================================
# DUPLICATE & CONSTANT COLUMN CHECK
# ==============================================================================

print("DUPLICATE EMPLOYEES CHECK")
print("=" * 65)
for key in ["general", "employee_survey", "manager_survey"]:
    df = raw[key]
    n_dup = df[EMPLOYEE_ID_COL].duplicated().sum()
    print(f"  {key}: {n_dup} duplicate EmployeeIDs {'✓' if n_dup == 0 else '⚠'}")

print("\nCONSTANT COLUMNS CHECK")
print("=" * 65)
df_gen = raw["general"]
constant_cols = [col for col in df_gen.columns if df_gen[col].nunique() <= 1]
if constant_cols:
    print(f"  Constant columns found (candidates for removal): {constant_cols}")
    for col in constant_cols:
        print(f"    → {col}: unique value = {df_gen[col].unique()}")
else:
    print("  No constant columns found.")

DUPLICATE EMPLOYEES CHECK
  general: 0 duplicate EmployeeIDs ✓
  employee_survey: 0 duplicate EmployeeIDs ✓
  manager_survey: 0 duplicate EmployeeIDs ✓

CONSTANT COLUMNS CHECK
  Constant columns found (candidates for removal): ['EmployeeCount', 'Over18', 'StandardHours']
    → EmployeeCount: unique value = [1]
    → Over18: unique value = <StringArray>
['Y']
Length: 1, dtype: str
    → StandardHours: unique value = [8]


In [6]:
# ==============================================================================
# EMPLOYEE ID CONSISTENCY ACROSS DATASETS
# ==============================================================================

print("EMPLOYEE ID CROSS-FILE CONSISTENCY")
print("=" * 65)

ids_general  = set(raw["general"][EMPLOYEE_ID_COL])
ids_employee = set(raw["employee_survey"][EMPLOYEE_ID_COL])
ids_manager  = set(raw["manager_survey"][EMPLOYEE_ID_COL])

# in_time / out_time: first column is EmployeeID (unnamed)
ids_in  = set(raw["in_time"].iloc[:, 0].astype(int))
ids_out = set(raw["out_time"].iloc[:, 0].astype(int))

all_sets = {
    "general": ids_general, "employee_survey": ids_employee,
    "manager_survey": ids_manager, "in_time": ids_in, "out_time": ids_out
}

ref = ids_general
for name, s in all_sets.items():
    only_ref  = ref - s
    only_this = s - ref
    if not only_ref and not only_this:
        print(f"  {name:20s} ↔ general: Perfect match ({len(s)} IDs) ✓")
    else:
        print(f"  {name:20s} ↔ general: {len(only_ref)} missing, {len(only_this)} extra ⚠")

EMPLOYEE ID CROSS-FILE CONSISTENCY
  general              ↔ general: Perfect match (4410 IDs) ✓
  employee_survey      ↔ general: Perfect match (4410 IDs) ✓
  manager_survey       ↔ general: Perfect match (4410 IDs) ✓
  in_time              ↔ general: Perfect match (4410 IDs) ✓
  out_time             ↔ general: Perfect match (4410 IDs) ✓


## Section 4: Badge Data Processing

**Purpose**: Transform raw badge timestamps (in_time / out_time) into meaningful features per employee:
- Average arrival & departure hours
- Average daily working hours
- Absence rate (% of working days with no badge)
- Punctuality indicators

In [7]:
# ==============================================================================
# BADGE DATA — Vectorized processing (fast)
# ==============================================================================
# CORRECTED: 
#   1. Restrict badge data to H1 2015 (Jan–Jun) to avoid late-year
#      disengagement contamination.
#   2. Drop avg_arrival_hour (quasi-constant, std=0.018) and
#      avg_departure_hour (r=0.9999 with avg_working_hours).

import gc

# --- Configuration: badge temporal window ---
BADGE_CUTOFF = pd.Timestamp("2015-06-30")   # Only use H1 2015 data

def process_badge_data(df_in, df_out, cutoff=BADGE_CUTOFF):
    """
    Transform raw badge in/out timestamps into employee-level features.
    Fully vectorized — processes all employees in seconds.
    
    Parameters
    ----------
    cutoff : pd.Timestamp or None
        If set, only use badge columns whose date <= cutoff.
        Set to None to use all available dates.
    
    Returns a DataFrame indexed by EmployeeID with:
      - avg_working_hours
      - absence_rate (fraction of working days with NA badge)
      - late_arrival_rate (fraction of days arriving after 10:00)
    """
    emp_ids = df_in.iloc[:, 0].astype(int)
    
    # Extract only date columns (skip EmployeeID column)
    in_dates = df_in.iloc[:, 1:]
    out_dates = df_out.iloc[:, 1:]
    
    # --- Temporal restriction ---
    if cutoff is not None:
        date_cols = []
        for c in in_dates.columns:
            try:
                dt = pd.to_datetime(c, errors="coerce")
                if dt is not pd.NaT and dt <= cutoff:
                    date_cols.append(c)
            except Exception:
                pass
        print(f"  Badge temporal filter: keeping {len(date_cols)}/{in_dates.shape[1]} "
              f"columns (up to {cutoff.date()})")
        in_dates = in_dates[date_cols]
        out_dates = out_dates[date_cols]
    
    # Convert all timestamps at once (NaN/NA become NaT)
    in_parsed = in_dates.apply(pd.to_datetime, errors="coerce")
    out_parsed = out_dates.apply(pd.to_datetime, errors="coerce")
    
    # Extract hours as float (hour + minute/60)
    in_hours = in_parsed.apply(lambda col: col.dt.hour + col.dt.minute / 60)
    out_hours = out_parsed.apply(lambda col: col.dt.hour + col.dt.minute / 60)
    
    # Free parsed datetime DataFrames (no longer needed)
    del in_parsed, out_parsed, in_dates, out_dates
    gc.collect()
    
    # Working hours per day
    work_hours = out_hours - in_hours
    
    # Count valid days (not NaN) per employee
    n_days = in_hours.shape[1]
    valid_mask = in_hours.notna() & out_hours.notna()
    n_present = valid_mask.sum(axis=1)
    n_absent = n_days - n_present
    
    # Late arrivals (after 10:00)
    late_mask = valid_mask & (in_hours >= 10)
    n_late = late_mask.sum(axis=1)
    
    # Compute aggregates
    avg_work = work_hours.where(valid_mask).mean(axis=1)
    
    # Free large intermediate DataFrames
    del in_hours, out_hours, work_hours, valid_mask, late_mask
    gc.collect()
    
    # NOTE: avg_arrival_hour dropped (quasi-constant, std=0.018, |r|=0.014)
    #       avg_departure_hour dropped (r=0.9999 with avg_working_hours)
    result = pd.DataFrame({
        EMPLOYEE_ID_COL: emp_ids.values,
        "avg_working_hours":  avg_work,
        "absence_rate":       n_absent / n_days,
        "late_arrival_rate":  (n_late / n_present).where(n_present > 0),
    })
    
    return result

print("Processing badge data (vectorized, H1 2015 only)...")
df_badge = process_badge_data(raw["in_time"], raw["out_time"])

# Free raw badge data — no longer needed
del raw["in_time"], raw["out_time"]
gc.collect()

print(f"Badge features computed for {len(df_badge)} employees.")
df_badge.describe().round(3)

Processing badge data (vectorized, H1 2015 only)...
  Badge temporal filter: keeping 129/261 columns (up to 2015-06-30)
Badge features computed for 4410 employees.


,EmployeeID,avg_working_hours,absence_rate,late_arrival_rate
count,4410.000,4410.000,4410.000,4410.000
mean,2205.500,7.701,0.088,0.501
std,1273.202,1.340,0.025,0.046
min,1.000,5.930,0.039,0.331
25%,1103.250,6.660,0.070,0.470
50%,2205.500,7.410,0.085,0.500
75%,3307.750,8.359,0.109,0.530
max,4410.000,11.047,0.186,0.672


## Section 5: Dataset Merge

**Purpose**: Inner-join all datasets on EmployeeID to create the unified analysis DataFrame.

In [8]:
# ==============================================================================
# MERGE ALL DATASETS
# ==============================================================================

df = raw["general"].copy()

# Merge survey data
df = df.merge(raw["employee_survey"], on=EMPLOYEE_ID_COL, how="left")
df = df.merge(raw["manager_survey"],  on=EMPLOYEE_ID_COL, how="left")
df = df.merge(df_badge,               on=EMPLOYEE_ID_COL, how="left")

# Drop constant / uninformative columns identified earlier
cols_to_drop = ["EmployeeCount", "Over18", "StandardHours"]
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])

print(f"Merged dataset: {df.shape[0]} rows x {df.shape[1]} columns")
print(f"Target distribution (Attrition):")
print(df["Attrition"].value_counts())
print(f"\nAttrition rate: {(df['Attrition'] == 'Yes').mean() * 100:.1f}%")

Merged dataset: 4410 rows x 29 columns
Target distribution (Attrition):
Attrition
No     3699
Yes     711
Name: count, dtype: int64

Attrition rate: 16.1%


In [9]:
# ==============================================================================
# FINAL QUALITY CHECK ON MERGED DATA
# ==============================================================================

print("POST-MERGE QUALITY CHECK")
print("=" * 65)

# Missing values in merged dataset
total_missing = df.isnull().sum()
cols_with_na = total_missing[total_missing > 0]
if cols_with_na.empty:
    print("  No missing values in merged dataset ✓")
else:
    print(f"  {len(cols_with_na)} columns with missing values:")
    for col, count in cols_with_na.items():
        print(f"    → {col}: {count} ({count/len(df)*100:.1f}%)")

# Data types summary
print(f"\nColumn types:")
print(f"  Numeric : {df.select_dtypes(include='number').shape[1]}")
print(f"  Object  : {df.select_dtypes(include='object').shape[1]}")
print(f"  Total   : {df.shape[1]}")

POST-MERGE QUALITY CHECK
  5 columns with missing values:
    → NumCompaniesWorked: 19 (0.4%)
    → TotalWorkingYears: 9 (0.2%)
    → EnvironmentSatisfaction: 25 (0.6%)
    → JobSatisfaction: 20 (0.5%)
    → WorkLifeBalance: 38 (0.9%)

Column types:
  Numeric : 22
  Object  : 7
  Total   : 29


## Section 6: Export

In [10]:
# ==============================================================================
# EXPORT CLEAN DATASET
# ==============================================================================

output_path = os.path.join(OUTPUT_DIR, "merged_data.csv")
df.to_csv(output_path, index=False)
print(f"Merged dataset exported to: {os.path.abspath(output_path)}")
print(f"Shape: {df.shape}")

# Also save a quick summary
summary_path = os.path.join(OUTPUT_DIR, "data_validation_summary.txt")
with open(summary_path, "w") as f:
    f.write(f"Data Validation Summary — {datetime.now():%Y-%m-%d %H:%M}\n")
    f.write(f"{'='*50}\n")
    f.write(f"Total employees: {len(df)}\n")
    f.write(f"Total features:  {df.shape[1]}\n")
    f.write(f"Attrition rate:  {(df['Attrition']=='Yes').mean()*100:.1f}%\n")
    f.write(f"Columns dropped: {cols_to_drop}\n")
    f.write(f"Badge features added: avg_working_hours, absence_rate, late_arrival_rate\n")
    f.write(f"Badge temporal window: H1 2015 (up to {BADGE_CUTOFF.date()})\n")
    f.write(f"Removed: avg_arrival_hour (quasi-constant), avg_departure_hour (redundant)\n")

print(f"Summary saved to: {os.path.abspath(summary_path)}")
print("\n✓ Pipeline complete — proceed to 02_EDA_Explorer.ipynb")

Merged dataset exported to: C:\Users\yanis\Documents\CESI\A5\AI Project\HumanForYou\outputs\merged_data.csv
Shape: (4410, 29)
Summary saved to: C:\Users\yanis\Documents\CESI\A5\AI Project\HumanForYou\outputs\data_validation_summary.txt

✓ Pipeline complete — proceed to 02_EDA_Explorer.ipynb


## Section 7: Data Leakage Diagnostic — Badge Features

**Purpose**: Verify that badge-derived features (avg_working_hours, absence_rate, etc.) do not encode attrition that already occurred during 2015. If employees left mid-year, their badge metrics would reflect their departure rather than predict it.

> **ALTAI Requirement 2 (Robustness)**: Model validity depends on features being genuinely predictive, not tautological.

In [11]:
# ==============================================================================
# DATA LEAKAGE DIAGNOSTIC — Badge Features vs Attrition
# ==============================================================================
# Badge data is now restricted to H1 2015 (Jan–Jun).
# avg_arrival_hour and avg_departure_hour have been removed upstream.

import scipy.stats as stats

merged = pd.read_csv(os.path.join(OUTPUT_DIR, "merged_data.csv"))
attrition_yes = merged[merged["Attrition"] == "Yes"]
attrition_no  = merged[merged["Attrition"] == "No"]

print("=" * 65)
print("DATA LEAKAGE DIAGNOSTIC — Badge Features vs Attrition")
print("(Badge restricted to H1 2015)")
print("=" * 65)

badge_cols = [c for c in ["avg_working_hours", "absence_rate", "late_arrival_rate"]
              if c in merged.columns]

leakage_alerts = []
for col in badge_cols:
    yes_vals = attrition_yes[col].dropna()
    no_vals  = attrition_no[col].dropna()
    if len(yes_vals) == 0 or len(no_vals) == 0:
        continue

    u_stat, p_val = stats.mannwhitneyu(yes_vals, no_vals, alternative="two-sided")
    effect_size = yes_vals.mean() - no_vals.mean()
    std_pooled = merged[col].std()
    cohens_d = effect_size / std_pooled if std_pooled > 0 else 0

    print(f"\n{col}:")
    print(f"  Mean (Attrition=Yes): {yes_vals.mean():.4f}")
    print(f"  Mean (Attrition=No):  {no_vals.mean():.4f}")
    print(f"  Δ = {effect_size:+.4f}  |  Cohen's d = {cohens_d:+.3f}  |  p = {p_val:.2e}")

    if p_val < 0.001 and abs(cohens_d) > 0.5:
        print(f"  ⚠️  WARNING: Potential data leakage — large effect + high significance")
        leakage_alerts.append(col)
    elif p_val < 0.05:
        print(f"  ℹ️  Statistically significant but small/medium effect — likely safe")

if leakage_alerts:
    print(f"\n{'='*65}")
    print(f"⚠️  {len(leakage_alerts)} feature(s) flagged for potential leakage:")
    for col in leakage_alerts:
        print(f"   - {col}")
    print("\n   → Run ablation test in Notebook 05 to quantify impact")
else:
    print(f"\n✅ No strong leakage signal detected in badge features (H1 2015)")

DATA LEAKAGE DIAGNOSTIC — Badge Features vs Attrition
(Badge restricted to H1 2015)

avg_working_hours:
  Mean (Attrition=Yes): 8.3170
  Mean (Attrition=No):  7.5825
  Δ = +0.7345  |  Cohen's d = +0.548  |  p = 8.36e-37
  ⚠️  WARNING: Potential data leakage — large effect + high significance

absence_rate:
  Mean (Attrition=Yes): 0.0867
  Mean (Attrition=No):  0.0883
  Δ = -0.0016  |  Cohen's d = -0.063  |  p = 1.62e-01

late_arrival_rate:
  Mean (Attrition=Yes): 0.4996
  Mean (Attrition=No):  0.5007
  Δ = -0.0010  |  Cohen's d = -0.023  |  p = 7.13e-01

⚠️  1 feature(s) flagged for potential leakage:
   - avg_working_hours

   → Run ablation test in Notebook 05 to quantify impact
